In [ ]:
from pathlib import Path
import pkg_resources as pkg
PATH_DEEPRANK_CORE = Path(pkg.resource_filename("deeprankcore", ""))
ROOT = PATH_DEEPRANK_CORE.parent
PATH_TEST = ROOT / "tests"
from deeprankcore.query import (
    QueryCollection,
    ProteinProteinInterfaceResidueQuery,
    SingleResidueVariantResidueQuery,
    ProteinProteinInterfaceAtomicQuery)
from deeprankcore.tools.target import compute_targets
from deeprankcore.DataSet import save_hdf5_keys
from deeprankcore.domain.aminoacidlist import alanine, phenylalanine
import glob
import os
import re
import sys
import h5py
import numpy as np

- Generating 1ATN_ppi.hdf5

In [ ]:
import warnings
from Bio import BiopythonWarning
with warnings.catch_warnings():
    warnings.simplefilter("ignore", BiopythonWarning)
    warnings.simplefilter("ignore", RuntimeWarning)

    ref_path = str(PATH_TEST / "data/ref/1ATN/1ATN.pdb")
    pssm_path1 = str(PATH_TEST / "data/pssm/1ATN/1ATN.A.pdb.pssm")
    pssm_path2 = str(PATH_TEST / "data/pssm/1ATN/1ATN.B.pdb.pssm")
    chain_id1 = "A"
    chain_id2 = "B"
    pdb_paths = [
        str(PATH_TEST / "data/pdb/1ATN/1ATN_1w.pdb"),
        str(PATH_TEST / "data/pdb/1ATN/1ATN_2w.pdb"),
        str(PATH_TEST / "data/pdb/1ATN/1ATN_3w.pdb"),
        str(PATH_TEST / "data/pdb/1ATN/1ATN_4w.pdb")]

    queries = QueryCollection()

    for pdb_path in pdb_paths:
        # Append data points
        targets = compute_targets(pdb_path, ref_path)
        queries.add(ProteinProteinInterfaceResidueQuery(
            pdb_path = pdb_path,
            chain_id1 = chain_id1,
            chain_id2 = chain_id2,
            targets = targets,
            pssm_paths = {
                chain_id1: pssm_path1,
                chain_id2: pssm_path2
            }
        ))

    # Generate graphs and save them in hdf5 files
    output_paths = queries.process(queries, process_count=1)



- Generating train.hdf5, valid.hdf5, test.hdf5

In [ ]:
# utility functions, we'll rewrite them
def getBestScorePdbs(pdb_models_folder):
	'''
	Takes as input the folder containing the .pdb models generated by Pandora, 
	and select the best one from the .tsv file.
	The .tsv file contains the models' scores and is located in the same pdbs' folder.
	Returns a list containing such pdbs' paths.
	'''
	pdbs_list = [fname.split('molpdf_DOPE.tsv')[0]+open(fname).read().split('\t')[0] \
			 for fname in glob.glob(f'{pdb_models_folder}/*/*/molpdf_DOPE.tsv')]
	return pdbs_list

def getPssms(pdbs_list):
    '''
    Takes as input a list containing the selected pdbs' paths.
    
    Returns two lists, containing pssms' files of MHCs and peptides, respectively. 
    '''
    pssm_m = [glob.glob(pdb.replace('models/', 'pssm_mapped/').replace(pdb.split('/')[-1], '') \
			 + 'pssm/*M*.pssm')[0] for pdb in pdbs_list]
    pssm_p = [glob.glob(pdb.replace('models/', 'pssm_mapped/').replace(pdb.split('/')[-1], '') \
			 +'pssm/*P*.pssm')[0] for pdb in pdbs_list]
    return pssm_m, pssm_p

def processCsv(csv_file_path):
	'''
	Takes as input the .csv file contaning {task} data.
	Returns lists of ids, alleles, peptides, scores, cluster.
	'''
	csv = open(csv_file_path).read().split('\n')[1:-1]
	csv = [i.split(',')for i in csv]
	csv_transposed = [[row[column] for row in csv]for column in range(len(csv[0]))]
	return csv_transposed[8], csv_transposed[0], csv_transposed[1], csv_transposed[2], csv_transposed[10]

def getPilotTargets(pdbs_list, csv_file_path):
    '''
	Takes as input the pdbs list and the .csv containing {task} data.
	Returns lists of {task} targets for each pdb in pdbs_list, and the corresponding clusters. 
	'''

    pattern = r'(BA[_]\w+)[.]\w+[.]pdb'
    # Finds BA_xxxxx id in each pdb path's, and uses it to retrieve the corresponding target
    # Then pdb_targets variable will contain the targets according to pdbs_list paths' order
    pdb_ids = re.compile(pattern)
    
    ids, _, _, scores, clusters = processCsv(csv_file_path)
    # Creates binary targets, keeping also the continuous value of {task} as second element of each sublist
    targets = [[int(float(score) <= 500), float(score)] for _, score in enumerate(scores)]
    pdb_targets = [targets[ids.index(pdb_ids.findall(pdb)[0])] \
        for pdb in pdbs_list]
    return pdb_targets, clusters

# Local data
project_folder = '/Users/giuliacrocioni/Desktop/docs/eScience/projects/3D-vac/snellius_50/'
distance_cutoff = 15 # max distance in Å between two interacting residues/atoms of two proteins

pdb_models_folder = f'{project_folder}data/pMHCI/models/BA/'
csv_file_path = f'{project_folder}data/binding_data/BA_pMHCI.csv'

print('Script running has started ...')
pdbs_list = getBestScorePdbs(pdb_models_folder)
print(f'pdbs files paths loaded, {len(pdbs_list)} pdbs found.')
pssm_m, pssm_p = getPssms(pdbs_list)
print(f'pssms files paths loaded, {len(pssm_m)} and {len(pssm_p)} files found for M and P, respectively.\n')
pdb_targets, clusters = getPilotTargets(pdbs_list, csv_file_path)
print(f'Targets retrieved from {csv_file_path}, and aligned with pdbs files.\n\
    There are {len(pdb_targets)} targets values and {len(clusters)} cluster values.\
    Total number of clusters is {len(set(clusters))}.\n')

queries - QueryCollection()
for it, pdb in enumerate(pdbs_list):
	queries.add(
		ProteinProteinInterfaceResidueQuery(
			pdb_path = pdb, 
			chain_id1 = "M",
			chain_id2 = "P",
			distance_cutoff = distance_cutoff,
			targets = {
				'binary': pdb_targets[it][0], # binary target value
				'BA': pdb_targets[it][1], # continuous target value
				'cluster': clusters[it]
				},
			pssm_paths = {
				"M": pssm_m[it],
				"P": pssm_p[it]
				})
	)
print(f'Queries created and ready to be processed.\n')

output_paths = queries.process(process_count = 1)
print(output_paths)

In [ ]:
# dividing hdf5 file in train, valid, test
hdf5_path = 'residue.hdf5'
train_clusters = [3, 4, 5, 2]
val_clusters = [1, 8]
test_clusters = [6]
target = 'target_values'
feature = 'cluster'

clusters = {}
train_ids = []
val_ids = []
test_ids = []
# '/Users/giuliacrocioni/remote_snellius/data/pMHCI/features_output_folder/GNN/residue/13072022/residue.hdf5'
with h5py.File(hdf5_path, 'r') as hdf5:

    for key in hdf5.keys():
        feature_value = float(hdf5[key][target][feature][()])
        if feature_value in train_clusters:
            train_ids.append(key)
        elif feature_value in val_clusters:
            val_ids.append(key)
        elif feature_value in test_clusters:
            test_ids.append(key)

        if feature_value in clusters.keys():
            clusters[int(feature_value)] += 1
        else:
            clusters[int(feature_value)] = 1


    print(f'Trainset contains {len(train_ids)} data points, {round(100*len(train_ids)/len(hdf5.keys()), 2)}% of the total data.')
    print(f'Validation set contains {len(val_ids)} data points, {round(100*len(val_ids)/len(hdf5.keys()), 2)}% of the total data.')
    print(f'Test set contains {len(test_ids)} data points, {round(100*len(test_ids)/len(hdf5.keys()), 2)}% of the total data.\n')

    for (key, value) in dict(sorted(clusters.items(), key=lambda x:x[1], reverse=True)).items():
        print(f'Group with value {key}: {value} data points, {round(100*value/len(hdf5.keys()), 2)}% of total data.')

save_hdf5_keys(hdf5_path, train_ids, 'train.hdf5', hardcopy = True)
save_hdf5_keys(hdf5_path, val_ids, 'valid.hdf5', hardcopy = True)
save_hdf5_keys(hdf5_path, test_ids, 'test.hdf5', hardcopy = True)

- Generating variants.hdf5

In [ ]:
count_queries = 5
pdb_path = str(PATH_TEST / "data/pdb/3C8P/3C8P.pdb")
ref_path = str(PATH_TEST / "data/ref/3C8P/3C8P.pdb")
targets = compute_targets(pdb_path, ref_path)
queries = QueryCollection()

for number in range(1, count_queries + 1):
    query = SingleResidueVariantResidueQuery(
        pdb_path,
        "A",
        number,
        None,
        alanine,
        phenylalanine,
        pssm_paths={
            "A": str(PATH_TEST / "data/pssm/3C8P/3C8P.A.pdb.pssm"),
            "B": str(PATH_TEST / "data/pssm/3C8P/3C8P.B.pdb.pssm")},
        targets = targets
    )
    queries.add(query)

output_paths = queries.add(queries, process_count = 1)

- Generating atomic.hdf5

In [ ]:
ref_path = str(PATH_TEST / "data/ref/1ATN/1ATN.pdb")
pssm_path1 = str(PATH_TEST / "data/pssm/1ATN/1ATN.A.pdb.pssm")
pssm_path2 = str(PATH_TEST / "data/pssm/1ATN/1ATN.B.pdb.pssm")
chain_id1 = "A"
chain_id2 = "B"
pdb_paths = [
    str(PATH_TEST / "data/pdb/1ATN/1ATN_1w.pdb"),
    str(PATH_TEST / "data/pdb/1ATN/1ATN_2w.pdb"),
    str(PATH_TEST / "data/pdb/1ATN/1ATN_3w.pdb"),
    str(PATH_TEST / "data/pdb/1ATN/1ATN_4w.pdb")]

queries = QueryCollection()

for pdb_path in pdb_paths:
    # Append data points
    targets = compute_targets(pdb_path, ref_path)
    queries.add(ProteinProteinInterfaceAtomicQuery(
        pdb_path = pdb_path,
        chain_id1 = chain_id1,
        chain_id2 = chain_id2,
        targets = targets,
        pssm_paths = {
            chain_id1: pssm_path1,
            chain_id2: pssm_path2
        }
    ))

# Generate graphs and save them in hdf5 files
output_paths = queries.process(process_count=1)